<a href="https://colab.research.google.com/github/faizanahemad/eva/blob/master/assignment-14/DN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tried**
- Alternate Augmentations
    - AutoAugment
    - Cutout
    - hue/saturation/brightness
- Used DataSets.map for building augmentors
- Used Slanted OLR with Slowdown
- GridSearch on Model+Training+Augmentation with Result storage in text file
- Momentum Scheduling like OLR.

**Results**:

```
{'train': 0.9651333333333333,
  'test': 0.9350333333333333,
  'training_time': 379.8242540359497,
  'augmentation_time': 769.7456877231598}
  
{'train': 0.9452866666666666,
  'test': 0.9206,
  'training_time': 172.4951605796814,
  'augmentation_time': 478.07853603363037},
```


**Resources**

- XLA

    - [Google Notebook example](https://colab.research.google.com/github/tensorflow/tensorflow/blob/master/tensorflow/compiler/xla/g3doc/tutorials/xla_compile.ipynb)

    - [Blog](https://medium.com/@xianbao.qian/use-xla-with-keras-3ca5d0309c26)
    - [Example Enabling XLA](https://github.com/tensorflow/models/blob/7212436440eaa11293ca84befcc5d8327109ea76/official/utils/misc/keras_utils.py#L158)
    
- Mixed Precision

- Augmentation Libs
    - [imgaug](https://github.com/aleju/imgaug)
    - [albumentations](https://github.com/albu/albumentations)
    - [Automold](https://github.com/UjjwalSaxena/Automold--Road-Augmentation-Library)
    - [Tensorflow Examples](https://www.wouterbulten.nl/blog/tech/data-augmentation-using-tensorflow-data-dataset/)
    - [PIL ImageOps Examples](https://hhsprings.bitbucket.io/docs/programming/examples/python/PIL/ImageOps.html)
    
- Tensorflow References
    - [Using Numpy functions](https://www.tensorflow.org/api_docs/python/tf/numpy_function)
    - [Using Python functions](https://www.tensorflow.org/api_docs/python/tf/py_function)
    - [Tensorflow Data](https://www.tensorflow.org/datasets/catalog/overview)
    - [Dataset to TFRecord](https://github.com/tensorflow/tensorflow/issues/16926)
    - [TFRecord](https://www.tensorflow.org/tutorials/load_data/tf_records#tfrecords_format_details)
    - [TFRecord Load](https://www.tensorflow.org/tutorials/load_data/images)

# Imports

In [2]:
import os
import gc

WEIGHT_DECAY = 1e-3 #@param {type:"number"}



jobs = int(os.cpu_count()/2)




In [ ]:
import tensorflow as tf

# tf.keras.backend.set_epsilon(1e-4)
# tf.keras.backend.set_floatx('float16')

tf.enable_eager_execution()
# tf.random.set_random_seed(1234)


In [4]:
import numpy as np
import skimage
print(np.__version__)
print(skimage.__version__)


import time, math
from tqdm import tqdm_notebook as tqdm

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.contrib.eager as tfe
import gc
print(tf.__version__)
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
% matplotlib inline
%config InlineBackend.figure_format='retina'

import copy
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from tensorflow.core.protobuf import rewriter_config_pb2
import os
from importlib import reload
import lib
reload(lib)
from lib import *
from data_science_utils import misc
from data_science_utils.vision.keras.regularizers import get_cutout_eraser

import pandas as pd


import imgaug.augmenters as iaa
from matplotlib import cm
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,RandomFog,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose, ChannelDropout, ChannelShuffle,RandomContrast
)

1.17.0
0.15.0


W0826 18:04:43.328066 140607039358784 deprecation_wrapper.py:119] From /home/ec2-user/SageMaker/eva/assignment-14/lib.py:24: The name tf.keras.backend.set_session is deprecated. Please use tf.compat.v1.keras.backend.set_session instead.

W0826 18:04:43.329121 140607039358784 deprecation_wrapper.py:119] From /home/ec2-user/SageMaker/eva/assignment-14/lib.py:26: The name tf.keras.backend.get_session is deprecated. Please use tf.compat.v1.keras.backend.get_session instead.



1.14.0


Using TensorFlow backend.


In [5]:
logger.info('This is a warning')


I0826 18:04:43.451470 140607039358784 <ipython-input-5-bf20c9c65a9e>:1] This is a warning


# GridSearch

In [6]:
def read_and_append_to_results(filename, result_object=None, read=False):
    from pathlib import Path
    import ast
    my_file = Path(filename)
    if my_file.is_file():
        results = misc.load_list_per_line(filename)
    else:
        results = []
    
    results = list(filter(lambda x:len(x)>2,results))
    
    if result_object is not None:
        results.append(result_object)
        misc.save_list_per_line(results, filename)
        
    if read:
        def lit_eval(r):
            try:
                return ast.literal_eval(r)
            except:
                return None
        return list(filter(lambda x: x is not None,map(lit_eval,results)))
    else:
        return

In [7]:
def run_model(model,opt,global_step,BATCH_SIZE,EPOCHS,train_data,x_test, y_test,len_train,log_test_acc=False):
    len_test = len(x_test)
    train_accs = []
    test_accs = []
    test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE).prefetch(len_test)


    

    logger.debug("Starting Training")
    t = time.time()
    for epoch in range(EPOCHS):
        tf.keras.backend.set_learning_phase(1)    
        train_loss = test_loss = train_acc = test_acc = 0.0
        train_set = train_data[epoch]
        for (x, y) in train_set:
            with tf.GradientTape() as tape:
                loss, correct = model(x, y)

            var = model.trainable_variables
            grads = tape.gradient(loss, var)
            for g, v in zip(grads, var):
                g += v * (WEIGHT_DECAY/(epoch+1)) * BATCH_SIZE
            opt.apply_gradients(zip(grads, var), global_step=global_step)
#             opt.apply_gradients(zip(grads, var))

            train_loss += loss.numpy()
            train_acc += correct.numpy()
            train_accs.append(train_acc / len_train)
        if log_test_acc or epoch==EPOCHS-1:    
            tf.keras.backend.set_learning_phase(0)
            for (x, y) in test_set:
                loss, correct = model(x, y)
                test_loss += loss.numpy()
                test_acc += correct.numpy()
            test_accs.append(test_acc / len_test)
            logger.debug(msg("epoch = %2s"%epoch,'||train=> loss: %.3f' %(train_loss / len_train), 'acc: %.3f' % (train_acc / len_train), '||val=> loss: %.3f' % (test_loss / len_test), 'val acc: %.3f' %(test_acc / len_test), '%.1fs'%(time.time() - t)))
        logger.debug(msg("trained for epoch = ",epoch,"train acc = ",train_accs[-1]))
            
    time_spent = time.time() - t
    logger.debug(msg("Train acc = ",train_accs[-1],"Test acc =",test_accs[-1],"Time Taken = ",time_spent))
    return train_accs[-1],test_accs[-1],time_spent
    
    


In [8]:
# da_config = {"max_augmentations_per_img":2,"augmentations":[{"augmentation_type":{"params,proba"}}]}

In [9]:
def build_augmenters(da_config,train_mean):
    
    cutout_config = da_config["cutout_config"]
    cutout_mapper = get_numpy_wrapper(get_cutout_eraser(**cutout_config),"cutout")
    augmentations = [hflip_mapper,cutout_mapper]
    if "cifar10_augs_config" in da_config:
        cifar10_augs_config = da_config["cifar10_augs_config"]
        if type(cifar10_augs_config)==dict:
            cifar10_mapper = get_numpy_wrapper(CIFAR10Policy(**cifar10_augs_config, fillcolor=tuple(train_mean.astype(int)), log=False),"AutoAug")
            augmentations.append(cifar10_mapper)
        else:
            assert type(cifar10_augs_config)==list
            for cfc in cifar10_augs_config:
                cifar10_mapper = get_numpy_wrapper(CIFAR10Policy(**cfc, fillcolor=tuple(train_mean.astype(int)), log=False),"AutoAug")
                augmentations.append(cifar10_mapper)
    if "hue_config" in da_config:
        hue_config = da_config["hue_config"]
        hue_mapper = get_hue_aug(hue_config["max_delta"])
        augmentations.append(hue_mapper)
    
    full_wrapper = get_multimapper(augmentations)
    logger.debug("Augmentation Functions Built")
    return full_wrapper


def process_full_augmentation_all_epochs(full_wrapper, EPOCHS,BATCH_SIZE, normalize, x_train, y_train):
    train_data = {}
    len_train = len(x_train)
    normalize = get_numpy_wrapper(normalize, Tout=tf.float32)
    t = time.time()
    for epoch in range(EPOCHS):
        train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).map(full_wrapper, num_parallel_calls=jobs).map(normalize, num_parallel_calls=jobs).shuffle(len_train).batch(len_train).prefetch(len_train)
        i = 0
        for x,y in train_set:
            xt = x.numpy()
            logger.debug(msg("X Shape = ",xt.shape[0], "Train Length =",len_train))
            assert x.numpy().shape[0]==len_train
            train_set = (xt,y.numpy())
            i = i+1
        assert i==1
        train_data[epoch] = train_set
        logger.debug(msg("Augmentation Epoch = ",epoch,"Time Spent = %.1f" % (time.time() - t)))
        
    train_sets = {}
    for epoch in range(EPOCHS):
        key = (epoch)%(max(train_data.keys())+1)
        train_set = train_data[key]
        train_set =tf.data.Dataset.from_tensor_slices(train_set).batch(BATCH_SIZE).prefetch(len_train).cache()
        train_sets[epoch] = train_set
    time_taken = time.time() - t
    logger.info(msg("Augmentation Done for EPOCHS = %s, Time Taken = %.1f"%(EPOCHS,time_taken)))
    return train_sets,time_taken
    
    

def fetch_cifar_data():
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    classes = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']
    len_train, len_test = len(x_train), len(x_test)
    y_train = y_train.astype('int64').reshape(len_train)
    y_test = y_test.astype('int64').reshape(len_test)
    train_mean = np.mean(x_train, axis=(0,1,2))
    train_std = np.std(x_train, axis=(0,1,2))

    def normalize(x):
        return ((x.astype('float32') - train_mean) / train_std).astype('float32')

    
    pad4 = lambda x: np.pad(x, [(0, 0), (4, 4), (4, 4), (0, 0)], mode='reflect')
    x_train = pad4(x_train)
    x_test = normalize(x_test)
    logger.info("Data Fetching Done")
    return x_train, y_train, x_test, y_test, classes,len_train,len_test, normalize, train_mean


     

def model_builder(model_config,EPOCHS,BATCH_SIZE, len_train):
    
    batches_per_epoch = len_train//BATCH_SIZE + 1
    model = FNet(**model_config["model"])
    
    enable_olr = model_config["optimizer"]["enable_olr"]
    enable_momentum = enable_olr
    max_lr = model_config["optimizer"]["max_lr"]
    max_momentum = model_config["optimizer"]["max_momentum"]
    global_step = tf.train.get_or_create_global_step()
    
    if enable_olr:
        mid_epoch = model_config["optimizer"]["mid_epoch"]
        start_lr = model_config["optimizer"]["start_lr"]
        
        end_lr = model_config["optimizer"]["end_lr"]
        pre_end_lr = model_config["optimizer"]["pre_end_lr"]
        pre_end_epoch = model_config["optimizer"]["pre_end_epoch"]
        
        enable_momentum = model_config["optimizer"]["enable_momentum"]
        if enable_momentum:
            min_momentum = model_config["optimizer"]["min_momentum"]
            momentum_schedule =  lambda t: np.interp([t], [0, mid_epoch, pre_end_epoch, EPOCHS], [max_momentum, min_momentum, max_momentum, max_momentum])[0] 
            momentum_func = lambda: momentum_schedule(global_step/batches_per_epoch)
            
            
    
        lr_schedule = lambda t: np.interp([t], [0, mid_epoch, pre_end_epoch, EPOCHS], [start_lr, max_lr, pre_end_lr, end_lr])[0] # LR = 0.75
        lr_func = lambda: lr_schedule(global_step/batches_per_epoch)/BATCH_SIZE

    opt = tf.train.MomentumOptimizer(learning_rate=lr_func if enable_olr else max_lr, momentum=momentum_func if enable_momentum else max_momentum, use_nesterov=True)
    logger.debug("Model Built")
    return model,opt,global_step




def runner(configs,cv=3):
    # fetch Data
    x_train, y_train, x_test, y_test, classes,len_train,len_test, normalize,train_mean = fetch_cifar_data()
    prev_da_config = None
    train_data = None
    results = []
    logger.info("Started GridSearch, Total Configs to Test = %s"%len(configs))
    reset_keras()
    t = time.time()
    prev_epochs = 0
    for i,config in enumerate(configs):
        
        _ = gc.collect()
        model_config = config["model_config"]
        da_config = config["augmentation_config"]
        BATCH_SIZE = config["training_config"]["BATCH_SIZE"]
        EPOCHS = config["training_config"]["EPOCHS"]
        save_file = config["training_config"]["save_file"]
        
        
        same_as_previous_config = prev_da_config == da_config and EPOCHS==prev_epochs
        prev_da_config = da_config
        prev_epochs = EPOCHS
        
        if not same_as_previous_config:
            augmenter = build_augmenters(da_config,train_mean)
            train_data,time_taken = process_full_augmentation_all_epochs(augmenter, EPOCHS,BATCH_SIZE, normalize, x_train, y_train)
            
        train_acc = test_acc = time_spent = 0.0
        for j in range(cv):
            model,opt,global_step = model_builder(model_config,EPOCHS,BATCH_SIZE, len_train)
            t1_acc,t2_acc,tsp = run_model(model,opt,global_step,BATCH_SIZE,EPOCHS,train_data,x_test, y_test,len_train,log_test_acc=False)
            train_acc += t1_acc
            test_acc += t2_acc
            time_spent += tsp
            reset_keras()
        train_acc /= cv
        test_acc /= cv
        time_spent /= cv
        
        logger.info(msg("Trial =",i,"Train acc = ",train_acc,"Test acc =",test_acc,"Time Taken = ",time_spent, "Model Params = %.2f"%(model.count_params()/1e6)))
            
        
        result = dict(config=config,results=dict(train=train_acc,test=test_acc,training_time=time_spent,augmentation_time=time_taken))
        read_and_append_to_results(save_file, result)
        results.append(result)
        del model
        model = None
    logger.info("Grid Search Complete, Total Results Count = %s, Time Taken = %.1f",len(results),(time.time()-t))
    assert len(results)==len(configs)
    return results
    



In [ ]:
# Testing with Less Epochs
augmentation_config = dict(cutout_config=dict(s_l=0.03, s_h=0.04,max_erasures_per_image=1,proba=0.5))

model_config = dict(model=dict(start_kernels=32,sparse_bn=True,thin_block=False,
                               enable_skip=True,enable_pool_before_skip=True, 
                               no_activation_first_conv=False, residual_dropout=0.0, spatial_dropout=0.05),
                    optimizer=dict(enable_olr=True,max_lr=0.75,
                                   start_lr=0.02,pre_end_lr=0.08,end_lr=0.02,
                                   mid_epoch=2,pre_end_epoch=6,
                                   max_momentum=0.9,min_momentum=0.8, enable_momentum=True,))

training_config = dict(BATCH_SIZE=512,EPOCHS=7,save_file="results-trails.txt")

trail_conf = dict(augmentation_config=augmentation_config,model_config=model_config,training_config=training_config)


results = runner([trail_conf],cv=3)

results

# 0.8744666666666667,0.8758333333333334 = gmp,
# 0.8753666666666667, 0.8725999999999999, 0.8727333333333332

In [13]:
augmentation_config = dict(cutout_config=dict(s_l=0.04, s_h=0.06,max_erasures_per_image=1),
                           hue_config=dict(max_delta=0.05),
                           cifar10_augs_config=dict(proba=0.5, enabled_policies=[("shearX",0.05, 0.1,),("shearY",0.05, 0.1,)]))

model_config = dict(model=dict(start_kernels=64,sparse_bn=True,thin_block=False,
                               enable_skip=True,enable_pool_before_skip=True, 
                               no_activation_first_conv=False, residual_dropout=0.0, spatial_dropout=0.0),
                    optimizer=dict(enable_olr=True,max_lr=0.6,
                                   start_lr=0.01,pre_end_lr=0.04,end_lr=0.02,
                                   mid_epoch=5,pre_end_epoch=14,
                                   max_momentum=0.9,min_momentum=0.8, enable_momentum=True,))

training_config = dict(BATCH_SIZE=512,EPOCHS=15,save_file="results.txt")

test_config_0 = dict(augmentation_config=augmentation_config,model_config=model_config,training_config=training_config)
test_config_0["augmentation_config"]["cifar10_augs_config"]["enabled_policies"] = [("shearX",0.05, 0.1,),("shearY",0.05, 0.1,)]
test_config_0["model_config"]["model"]["spatial_dropout"] = 0.02
test_config_0["model_config"]["model"]["sparse_bn"] = False
test_config_0["training_config"]["EPOCHS"] = 24
test_config_0["model_config"]["optimizer"]["mid_epoch"] = 6
test_config_0["model_config"]["optimizer"]["pre_end_epoch"] = 22
test_config_0["model_config"]["optimizer"]["max_lr"] = 0.7
test_config_0["model_config"]["optimizer"]["min_momentum"] = 0.7

test_config_1 = copy.deepcopy(test_config_0)
test_config_1["model_config"]["optimizer"]["mid_epoch"] = 4



results = runner([test_config_0,test_config_1],cv=3)




I0826 18:12:47.283362 140607039358784 <ipython-input-9-893880cce488>:72] Data Fetching Done
I0826 18:12:47.284206 140607039358784 <ipython-input-9-893880cce488>:121] Started GridSearch, Total Configs to Test = 2
I0826 18:25:32.231678 140607039358784 <ipython-input-9-893880cce488>:51]  Augmentation Done for EPOCHS = 24, Time Taken = 764.7
W0826 18:25:36.728163 140607039358784 deprecation.py:323] From /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/keras/backend.py:3940: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
I0826 18:44:34.168587 140607039358784 <ipython-input-9-893880cce488>:155]  Trial = 0 Train acc =  0.9626733333333334 Test acc = 0.9330333333333334 Time Taken =  380.04483993848163 Model Params = 6.60
I0826 19:03:31.984951 140607039358784 <ipython-input-9-89

# Reading Results from Text File

In [12]:
len(read_and_append_to_results("results.txt", read=True))
[x["results"] for x in read_and_append_to_results("results.txt", read=True)[-20:]]

# Augmentations per image

162

[{'train': 0.9301733333333333,
  'test': 0.9179333333333334,
  'training_time': 199.6779501438141,
  'augmentation_time': 447.832466840744},
 {'train': 0.9244533333333335,
  'test': 0.9194,
  'training_time': 210.19939549763998,
  'augmentation_time': 496.33520460128784},
 {'train': 0.9313733333333333,
  'test': 0.9208,
  'training_time': 195.69241094589233,
  'augmentation_time': 478.4510440826416},
 {'train': 0.9452866666666666,
  'test': 0.9206,
  'training_time': 172.4951605796814,
  'augmentation_time': 478.07853603363037},
 {'train': 0.9315066666666668,
  'test': 0.9198333333333334,
  'training_time': 198.11389915148416,
  'augmentation_time': 478.07853603363037},
 {'train': 0.9396800000000001,
  'test': 0.9222,
  'training_time': 197.06325793266296,
  'augmentation_time': 478.07853603363037},
 {'train': 0.9441333333333333,
  'test': 0.9175000000000001,
  'training_time': 172.49557248751321,
  'augmentation_time': 478.07853603363037},
 {'train': 0.94476,
  'test': 0.9200333333333